# Stockpicking Template
This Pyhton script aims ti provided profunded and solid investment advice. Especially in times where we get tons of informatins and "Financial Influencers" want to tell us which stocks to hold it is important to hav tools at hand to filter the "noise" and come to an good and unbiased investment advice.

The skript is based on a jupyter notebook by Nick McCullum. Learn more about Nick here: https://nickmccullum.com/
https://www.youtube.com/watch?v=xfzGZB4HhEE&t=13248s. The sections about the API Calls and the inputs will be in line with his course on "Algorithmic Trading Using Python - Full Course" (https://www.youtube.com/watch?v=xfzGZB4HhEE&t=13248s)

In here, I will build a programm which only needs a excel sheet as an input to perform valueable evaluating and screening for us to determine based on our rpeferences wich stocks are worth a closer look and from which stocks to stay away.



## Library Imports
The first thing we need to do is import libaries and functions that I put in a `functions.py` file

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats #The SciPy stats module

from functions import chunks #for splitting number of stocks for batch API calls

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Parsing Our API Call
This API call has the metrics we need, for example:

In [3]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data
pe_ratio = data['peRatio']
pe_ratio

29.94

## Executing A Batch API Call & Building Our DataFrame
This is much faster than the calls we did before. The function chunks that we use divides our list of securities into groups of 100 which is the maximum amount of Tickers that we are allowed to use for one call.

In [4]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

## Collecting the metrics we are interested in

Every valuation metric has certain flaws. For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings such as Growth Stocks, which are stocks that I am particular interested in! Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

##### Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. 

In this section as it stands, we will filter for stocks with the lowest percentiles on the following metrics which represents a classical Value strategie:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

As it develops, the aim of this project is to have several strategies in several seperate `.py` files, so the user can use a certain strategie but also customnize his own strategie with the use of the others as template.

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [35]:
# Historische Daten ziehen ---> geht nicht wirklich mit batch api calls! Also normale benutzen!
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=chart&range=1y&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# What metrics are possible to gather? # endpoints fundamentals
data
# For more informations visit the documentation of IEX Cloud!

{'AAPL': {'chart': [{'close': 80.8633,
    'high': 81.9,
    'low': 79.479,
    'open': 82.795,
    'symbol': 'AAPL',
    'volume': 128134244,
    'id': 'PLISRIIAESROCHCT_',
    'key': 'ALAP',
    'subkey': '',
    'date': '2020-05-26',
    'updated': 1647660247652,
    'changeOverTime': 0,
    'marketChangeOverTime': 0,
    'uOpen': 334.4,
    'uClose': 319.7,
    'uHigh': 327.14,
    'uLow': 318.4,
    'uVolume': 32151346,
    'fOpen': 80.4327,
    'fClose': 78.799,
    'fHigh': 84.4619,
    'fLow': 79.0293,
    'fVolume': 130415906,
    'label': 'May 26, 20',
    'change': 0,
    'changePercent': 0},
   {'close': 81.6582,
    'high': 80.812,
    'low': 79.2004,
    'open': 81.602,
    'symbol': 'AAPL',
    'volume': 114619977,
    'id': 'CIHEOASSTL_PIRCIR',
    'key': 'LPAA',
    'subkey': '',
    'date': '2020-05-27',
    'updated': 1680852174144,
    'changeOverTime': 0.004430575669207284,
    'marketChangeOverTime': 0.004486279554959341,
    'uOpen': 322.13,
    'uClose': 331.9,


In [9]:
# Historische Daten ziehen für Umsatzwachstum. FCF Wachstum, usw

symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/time-series/FUNDAMENTALS/AAPL/10-K?range=5y&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
data
# What metrics are possible to gather?




[]

In [14]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# What metrics are possible to gather?
data[symbol]['advanced-stats']
# For more informations visit the documentation of IEX Cloud!

{'beta': 1.420174343770394,
 'totalCash': 72014557964,
 'currentDebt': 109409585115,
 'revenue': 335529853273,
 'grossProfit': 132927568809,
 'totalRevenue': 327740529179,
 'EBITDA': 101379448969,
 'revenuePerShare': 20.3,
 'revenuePerEmployee': 2316061.24,
 'debtToEquity': 5.048181358776389,
 'profitMargin': 0.2460296133578526,
 'enterpriseValue': 2199779530990,
 'enterpriseValueToRevenue': 6.8,
 'priceToSales': 6.83,
 'priceToBook': 31.53,
 'forwardPERatio': 24.671985958307417,
 'pegRatio': 0.7305016958477157,
 'peHigh': 13.71316057802822,
 'peLow': 10.225677004422277,
 'week52highDate': '2021-01-23',
 'week52lowDate': '2020-05-13',
 'putCallRatio': 0.4164946692545272,
 'companyName': 'Apple Inc',
 'marketcap': 2162486405091,
 'week52high': 147.97,
 'week52low': 79.74,
 'week52highSplitAdjustOnly': 152.34,
 'week52highDateSplitAdjustOnly': '2021-01-13',
 'week52lowSplitAdjustOnly': 78.28,
 'week52lowDateSplitAdjustOnly': '2020-05-12',
 'week52change': 0.6765781710154415,
 'sharesOuts

##### Lets get a feeling for this!

In [6]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit



### Lets build it!


In [7]:
rv_columns = [
    'Ticker',
    'Price',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
      # include more try expect statements!
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
rv_dataframe.head()

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,138.55,54,N/A,8.85,N/A,7.69,N/A,32.380926,N/A,14.329241,N/A,N/A
1,AAL,23.78,-1.63,N/A,-1.91,N/A,1.17,N/A,-3.690478,N/A,3.132654,N/A,N/A
2,AAP,194.00,27.61,N/A,3.68,N/A,1.27,N/A,12.625574,N/A,2.859598,N/A,N/A
3,AAPL,128.30,28.95,N/A,31.03,N/A,6.73,N/A,21.234433,N/A,16.641867,N/A,N/A
4,ABBV,116.71,40.99,N/A,15.26,N/A,4.17,N/A,13.491222,N/A,8.927551,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

Brief explaination to the following code:
+ This will first check with isnull() wheter we have a None at the position in the df and then we will apply any(axis=1) to specify that we want to collect any row (axis=1) where we have True value that indicates a np.nan

In [8]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)].head()

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,263.66,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,299.17,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,9.75,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,29.92,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.81,14.58,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


### Missing data

Dealing with missing data is an important topic in data science and of course in this matter of investing

I decided to drop all missing values using dropna(), since it does no make sense to use averages or other commen imputation methods!

In [9]:
rv_dataframe.dropna(axis=0, inplace=True, subset=["Ticker", "Price", 'Price-to-Earnings Ratio', 'Price-to-Book Ratio',
                                                 'Price-to-Sales Ratio','EV/EBITDA','EV/GP'])

In [10]:
# this should be empty now!

rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [15]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0     0.44
1     0.08
2      0.3
3     0.02
4     0.28
5     0.14
6      0.6
7     0.36
8     0.32
9     0.46
10    0.26
11     0.7
12    0.06
13    0.34
14    0.52
15    0.74
16    0.89
17    0.68
18    0.89
19    0.42
20    0.94
21    0.58
22    0.78
23    0.96
24    0.62
25    0.22
26    0.48
27       1
28    0.38
29     0.4
30    0.84
31    0.56
32    0.72
33     0.2
34    0.98
35    0.04
36     0.5
37    0.12
38    0.16
39    0.92
40    0.18
41    0.24
42     0.1
43    0.82
44    0.64
45    0.76
46    0.54
47     0.8
48    0.66
49    0.86
Name: PE Percentile, dtype: object
0     0.12
1     0.86
2     0.02
3      0.2
4     0.14
5     0.28
6     0.22
7      0.1
8     0.04
9     0.56
10    0.08
11    0.78
12    0.43
13    0.75
14    0.71
15    0.62
16    0.52
17    0.32
18    0.59
19    0.36
20    0.16
21    0.18
22    0.06
23    0.68
24    0.94
25    0.24
26    0.59
27    0.71
28     0.8
29    0.39
30    0.88
31    0.34
32    0.48
33       1
34    0.66
35    0.84
36    0.96
37    0.

C:\Users\Jonathan\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.493,7.92,0.44,0.6037,0.12,0.467,0.18,3.995255,0.18,0.474984,0.06,0.0572016
1,AIV,7.060,-77.43,0.08,2.03,0.86,-31.91,0.02,-45.365852,0.02,-55.558529,0.02,0.0623457
2,MCK,198.770,-7.03,0.3,-1535.41,0.02,0.1353,0.06,7.714043,0.54,2.669701,0.64,0.0674897
3,L,58.640,-434.69,0.02,0.9348,0.2,1.11,0.5,8.662011,0.68,1.064961,0.18,0.0687243
4,AIG,52.474,-11.47,0.28,0.7384,0.14,1.07,0.48,6.776743,0.36,1.030300,0.16,0.0703704
5,HFC,34.567,-35.48,0.14,1.07,0.28,0.4956,0.2,8.501338,0.64,2.947199,0.72,0.081893
6,FTI,8.350,11.06,0.6,1,0.22,0.3275,0.1,3.663273,0.16,3.414341,0.8,0.0942387
7,AAL,23.780,-1.63,0.36,-1.91,0.1,1.17,0.54,-3.690478,0.08,3.132654,0.76,0.0950617
8,ABC,119.880,-6.36,0.32,-87.58,0.04,0.127,0.04,8.721190,0.7,4.381737,0.9,0.0979424
9,VIAC,42.052,9.14,0.46,1.43,0.56,1.05,0.46,7.272218,0.46,1.465859,0.34,0.104321


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [16]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

C:\Users\Jonathan\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.493,7.92,0.44,0.6037,0.12,0.467,0.18,3.995255,0.18,0.474984,0.06,0.196
1,AIV,7.060,-77.43,0.08,2.03,0.86,-31.91,0.02,-45.365852,0.02,-55.558529,0.02,0.2
2,MCK,198.770,-7.03,0.3,-1535.41,0.02,0.1353,0.06,7.714043,0.54,2.669701,0.64,0.312
3,L,58.640,-434.69,0.02,0.9348,0.2,1.11,0.5,8.662011,0.68,1.064961,0.18,0.316
4,AIG,52.474,-11.47,0.28,0.7384,0.14,1.07,0.48,6.776743,0.36,1.030300,0.16,0.284
5,HFC,34.567,-35.48,0.14,1.07,0.28,0.4956,0.2,8.501338,0.64,2.947199,0.72,0.396
6,FTI,8.350,11.06,0.6,1,0.22,0.3275,0.1,3.663273,0.16,3.414341,0.8,0.376
7,AAL,23.780,-1.63,0.36,-1.91,0.1,1.17,0.54,-3.690478,0.08,3.132654,0.76,0.368
8,ABC,119.880,-6.36,0.32,-87.58,0.04,0.127,0.04,8.721190,0.7,4.381737,0.9,0.4
9,VIAC,42.052,9.14,0.46,1.43,0.56,1.05,0.46,7.272218,0.46,1.465859,0.34,0.456


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [13]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)


In [73]:
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Number of Shares to Buy
0,UNM,31.490,8.31,0.147541,0.62,0.0532787,0.4884,0.0204918,4.103417,0.0512295,0.474915,0.00819672,0.0561475,63.0
1,MCK,206.255,-7.16,0.114754,-1500.02,0.00409836,0.1317,0.00614754,7.510971,0.0942623,2.616650,0.10041,0.0639344,9.0
2,L,58.990,-437.33,0.00409836,0.917,0.0696721,1.14,0.103484,8.635573,0.127049,1.046217,0.0286885,0.0665984,33.0
3,AIG,53.700,-11.65,0.0963115,0.7177,0.057377,1.04,0.0881148,6.652337,0.0696721,1.006573,0.0245902,0.0672131,37.0
4,HFC,36.640,-40.24,0.0512295,1.13,0.0840164,0.5246,0.022541,8.826671,0.135246,3.029586,0.127049,0.0840164,54.0
5,ABC,122.000,-6.72,0.118852,-88.08,0.00819672,0.1247,0.00409836,8.614181,0.122951,4.142675,0.20082,0.0909836,16.0
6,AAL,23.420,-1.6,0.135246,-1.82,0.0512295,1.13,0.0983607,-3.780446,0.0389344,3.105785,0.131148,0.0909836,85.0
7,VIAC,39.864,8.9,0.151639,1.37,0.106557,0.9947,0.079918,7.022064,0.0737705,1.419952,0.045082,0.0913934,50.0
8,FTI,8.731,11.21,0.184426,1.04,0.0737705,0.3237,0.0102459,3.621668,0.0471311,3.452640,0.143443,0.0918033,229.0
9,F,12.219,12.72,0.204918,1.42,0.122951,0.3688,0.0163934,3.943253,0.0491803,2.287911,0.0881148,0.0963115,163.0


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [65]:
portfolio_input()

Enter the value of your portfolio:100000


In [69]:
position_size = float(portfolio_size) / len(rv_dataframe.index);
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Number of Shares to Buy
0,UNM,31.490,8.31,0.147541,0.62,0.0532787,0.4884,0.0204918,4.103417,0.0512295,0.474915,0.00819672,0.0561475,63.0
1,MCK,206.255,-7.16,0.114754,-1500.02,0.00409836,0.1317,0.00614754,7.510971,0.0942623,2.616650,0.10041,0.0639344,9.0
2,L,58.990,-437.33,0.00409836,0.917,0.0696721,1.14,0.103484,8.635573,0.127049,1.046217,0.0286885,0.0665984,33.0
3,AIG,53.700,-11.65,0.0963115,0.7177,0.057377,1.04,0.0881148,6.652337,0.0696721,1.006573,0.0245902,0.0672131,37.0
4,HFC,36.640,-40.24,0.0512295,1.13,0.0840164,0.5246,0.022541,8.826671,0.135246,3.029586,0.127049,0.0840164,54.0
5,ABC,122.000,-6.72,0.118852,-88.08,0.00819672,0.1247,0.00409836,8.614181,0.122951,4.142675,0.20082,0.0909836,16.0
6,AAL,23.420,-1.6,0.135246,-1.82,0.0512295,1.13,0.0983607,-3.780446,0.0389344,3.105785,0.131148,0.0909836,85.0
7,VIAC,39.864,8.9,0.151639,1.37,0.106557,0.9947,0.079918,7.022064,0.0737705,1.419952,0.045082,0.0913934,50.0
8,FTI,8.731,11.21,0.184426,1.04,0.0737705,0.3237,0.0102459,3.621668,0.0471311,3.452640,0.143443,0.0918033,229.0
9,F,12.219,12.72,0.204918,1.42,0.122951,0.3688,0.0163934,3.943253,0.0491803,2.287911,0.0881148,0.0963115,163.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [22]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [23]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [24]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [25]:
writer.save()